In [125]:
from meteostat import Stations, Daily
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [126]:
stations = Stations()
stations = stations.nearby(51.5072, 0.1276) # London
station = stations.fetch(1)

In [127]:
now = datetime.now()
now = datetime(now.year, now.month, now.day)
prev = now - (timedelta(days=30) * 2)
data = Daily(station, start=prev, end=now, model=False)
data = data.fetch().reset_index()

In [128]:
data.head()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2021-11-19,11.7,11.0,12.0,NaN,NaN,247.4,10.0,NaN,1030.5,NaN
1,2021-11-20,10.7,10.0,11.0,NaN,NaN,251.6,11.7,NaN,1023.5,NaN
2,2021-11-21,7.3,5.0,10.0,NaN,NaN,334.8,14.8,NaN,1019.6,NaN
3,2021-11-22,6.5,4.0,10.0,NaN,NaN,353.4,9.4,NaN,1030.1,NaN
4,2021-11-23,5.7,3.0,9.0,NaN,NaN,277.7,5.3,NaN,1032.1,NaN


In [129]:
df = data[["time", "tavg"]]

In [130]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

In [131]:
X = df["tavg"].values
X = X.reshape(len(X), 1) # make it vertical

In [132]:
scaler = scaler.fit(X)
scaled_X = pd.Series(scaler.transform(X)[:, 0])
scaled_X.head()

0    0.560976
1    0.398374
2   -0.154472
3   -0.284553
4   -0.414634
dtype: float64

In [133]:
training = df.copy()
training["tavg"] = scaled_X

In [139]:
# get 7 days before each point
days = 7

prev = []
for i in range(0, len(df)):
    prev.append(training.loc[i-days:i]["tavg"].values)

training["prev"] = prev
tdf = training.drop(range(7), inplace=False).reset_index(drop=True)

In [141]:
tdf.head()

,time,tavg,prev
0,2021-11-26,-0.382114,"[0.5609756097560974, 0.39837398373983723, -0.1..."
1,2021-11-27,-0.674797,"[0.39837398373983723, -0.15447154471544722, -0..."
2,2021-11-28,-0.934959,"[-0.15447154471544722, -0.2845528455284554, -0..."
3,2021-11-29,-1.000000,"[-0.2845528455284554, -0.41463414634146345, -0..."
4,2021-11-30,0.317073,"[-0.41463414634146345, -0.43089430894308955, -..."
